Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [43]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [44]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [45]:
df['date'] = pd.to_datetime(df['created'])
df.index = df['date']
print(df.info)
print(df.head())

<bound method DataFrame.info of                      bathrooms  ...                date
date                            ...                    
2016-06-24 07:54:24        1.5  ... 2016-06-24 07:54:24
2016-06-12 12:19:27        1.0  ... 2016-06-12 12:19:27
2016-04-17 03:26:41        1.0  ... 2016-04-17 03:26:41
2016-04-18 02:22:02        1.0  ... 2016-04-18 02:22:02
2016-04-28 01:32:41        1.0  ... 2016-04-28 01:32:41
...                        ...  ...                 ...
2016-06-02 05:41:05        1.0  ... 2016-06-02 05:41:05
2016-04-04 18:22:34        1.0  ... 2016-04-04 18:22:34
2016-04-16 02:13:40        1.0  ... 2016-04-16 02:13:40
2016-04-08 02:13:33        1.0  ... 2016-04-08 02:13:33
2016-04-12 02:48:07        1.0  ... 2016-04-12 02:48:07

[48817 rows x 35 columns]>
                     bathrooms  ...                date
date                            ...                    
2016-06-24 07:54:24        1.5  ... 2016-06-24 07:54:24
2016-06-12 12:19:27        1.0  ... 2016-06-

In [52]:
X = df[['bedrooms', 'bathrooms']]
y = df['price']

mask = X.index.month < 6

X_train = X[mask]
y_train = y[mask]
X_test = X[~mask]
y_test = y[~mask]

print(X_train)
print(y_train)
print(X_test)
print(y_test)

                     bedrooms  bathrooms
date                                    
2016-04-17 03:26:41         1        1.0
2016-04-18 02:22:02         1        1.0
2016-04-28 01:32:41         4        1.0
2016-04-19 04:24:47         4        2.0
2016-04-27 03:19:56         2        1.0
...                       ...        ...
2016-04-22 15:44:11         1        1.0
2016-04-04 18:22:34         1        1.0
2016-04-16 02:13:40         1        1.0
2016-04-08 02:13:33         0        1.0
2016-04-12 02:48:07         2        1.0

[31844 rows x 2 columns]
date
2016-04-17 03:26:41    2850
2016-04-18 02:22:02    3275
2016-04-28 01:32:41    3350
2016-04-19 04:24:47    7995
2016-04-27 03:19:56    3600
                       ... 
2016-04-22 15:44:11    4500
2016-04-04 18:22:34    3950
2016-04-16 02:13:40    2595
2016-04-08 02:13:33    3350
2016-04-12 02:48:07    2200
Name: price, Length: 31844, dtype: int64
                     bedrooms  bathrooms
date                                    
2016-

In [47]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

print(model.coef_)
print(model.intercept_)

[ 389.3248959  2072.61011639]
485.71869002322273


In [48]:
print('Train R squared value:', model.score(X_train, y_train))
print('Test R squared value:', model.score(X_test, y_test))

Train R squared value: 0.5111543084316605
Test R squared value: 0.5213303957090346


In [50]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print('Train MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))
print()
print('Train RMSE:', mean_squared_error(y_train, model.predict(X_train), squared=False))
print('Test RMSE:', mean_squared_error(y_test, model.predict(X_test), squared=False))

Train MAE: 818.5310213271714
Test MAE: 825.8987822403527

Train RMSE: 1232.0225917223486
Test RMSE: 1219.719357233823
